<a href="https://colab.research.google.com/github/ishit-k9/Classification-Machine-Learning/blob/main/Maths_Questions_Classification_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# **MATHEMATICS QUESTIONS CLASSIFICATION**
---

* **FRAMING THE PROBLEM**

Given a math problem written in natural language, your goal is to predict its correct topic from eight predefined categories. These include algebra, calculus, linear algebra, and more.

* **EVALUATION METRICS**

f1- score is defined as evaluation metrics for this problem.

* **DATA DESCRIPTION**

Dataset Description
Your task is to classify each Math problem into one of these eight topics using a machine learning or NLP-based approach.

Label	Topic

0	Algebra

1	Geometry and Trigonometry

2	Calculus and Analysis

3	Probability and Statistics

4	Number Theory

5	Combinatorics and Discrete Math

6	Linear Algebra

7	Abstract Algebra and Topology

# **IMPORT LIBRARIES**

In [ ]:
import numpy as np
import pandas as pd
import string
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, VotingClassifier, StackingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.metrics import f1_score, classification_report
from xgboost import XGBClassifier
from sklearn.svm import SVC

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/NLP practice datasets/Maths NLP Train.csv')
test = pd.read_csv('/content/drive/MyDrive/NLP practice datasets/Maths NLP Test.csv')

In [ ]:
train.head()

,Question,label
0,A solitaire game is played as follows. Six di...,3
1,2. The school table tennis championship was he...,5
2,"Given that $x, y,$ and $z$ are real numbers th...",0
3,$25 \cdot 22$ Given three distinct points $P\l...,1
4,I am thinking of a five-digit number composed ...,5


In [ ]:
train.shape

(10189, 2)

In [ ]:
train.isnull().sum()

,0
Question,0
label,0


In [ ]:
train.duplicated().sum()

np.int64(0)

In [ ]:
train['Question'][98]

'A circle of radius $10$ inches has its center at the vertex $C$ of an equilateral triangle $ABC$ and\npasses through the other two vertices. The side $AC$ extended through $C$ intersects the circle\nat $D$. The number of degrees of angle $ADB$ is:\n$\\text{(A)} 15 \\quad  \\text{(B)}30 \\quad \\text{(C)}60 \\quad \\text{(D)}90 \\quad \\text{(E)}120$'

In [ ]:
train['Question'][456]

'6. The line segment connecting two points on a sphere is called a chord of the sphere.\n\nFor a sphere with a radius of 4, the lengths of two chords \\( AB \\) and \\( CD \\) are \\( 2 \\sqrt{7} \\) and \\( 4 \\sqrt{3} \\) respectively. \\( M \\) and \\( N \\) are the midpoints of \\( AB \\) and \\( CD \\) respectively, and the endpoints of each pair of chords can move on the surface of the sphere. Given the following four propositions:\n(1) Chords \\( AB \\) and \\( CD \\) may intersect at point \\( M \\);\n(2) Chords \\( AB \\) and \\( CD \\) may intersect at point \\( N \\);\n(3) The maximum value of \\( MN \\) is 5;\n(4) The minimum value of \\( MN \\) is 1.\nAmong these, the true propositions are ( ).\n(A) (1)(3)(4)\n(B) (1)(2)(3)\n(C) (1)(2)(4)\n(D) (2)(3)(4)'

In [ ]:
train['Question'][7634]

'How many base-10 integers are exactly 4 digits in their base-3 representation and exactly 2 digits in their base-6 representation?'

In [ ]:
exclude = string.punctuation
exclude

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
# train[train['label'] == 3]
train['Question'][10134]

'Scientists were letting rats into a maze and observed whether they would reach the goal. They found that $56\\%$ of the black rats and $84\\%$ of the white rats reached the goal. The ratio of the number of black and white rats at the goal was $1:2$.\n\nWhat was the ratio of the number of black and white rats at the start?\n\n(M. Petrová)'

# **Is Greek letters present in Questions?**

In [ ]:
greek_letters = ['α', 'β', 'γ', 'δ', 'ε', 'ζ', 'η', 'θ', 'ι', 'κ', 'λ', 'μ', 'ν', 'ξ', 'ο', 'π', 'ρ', 'σ', 'τ', 'υ', 'φ', 'χ', 'ψ', 'ω',
                  'Γ', 'Δ', 'Θ',  'Λ',  'Ξ', 'Ο', 'Π', 'Σ',  'Φ',  'Ψ', 'Ω']
found_greek = False

for index, row in train.iterrows():
    for cell in row:
        if isinstance(cell, str):  # Ensure we're checking strings
            for letter in greek_letters:
                if letter in cell:
                    print(f"Found Greek letter '{letter}' in row {index}, column '{row.index[row.tolist().index(cell)]}': '{cell}'")
                    found_greek = True

if not found_greek:
    print("No Greek letters found in the dataset.")

Found Greek letter 'α' in row 1803, column 'Question': '10th Eötvös 1903 Problem 2 α and β are real and a = sin α, b = sin β, c = sin(α+β). Find a polynomial p(x, y, z) with integer coefficients, such that p(a, b, c) = 0. Find all values of (a, b) for which there are less than four distinct values of c.'
Found Greek letter 'β' in row 1803, column 'Question': '10th Eötvös 1903 Problem 2 α and β are real and a = sin α, b = sin β, c = sin(α+β). Find a polynomial p(x, y, z) with integer coefficients, such that p(a, b, c) = 0. Find all values of (a, b) for which there are less than four distinct values of c.'
Found Greek letter 'π' in row 3247, column 'Question': '41st Putnam 1980 Problem A3 Find ∫ 0 π/2 f(x) dx, where f(x) = 1/(1 + tan √2 x). Solution'
Found Greek letter 'α' in row 3984, column 'Question': '51st Putnam 1990 Problem B6 C is a non-empty, closed, bounded, convex subset of the plane. Given a support line L of C and a real number 0 ≤ α ≤ 1, let B α be the band parallel to L, si

In [ ]:
def newregex(text):
  text = re.sub(r'α', 'alpha', text)
  text = re.sub(r'β', 'beta', text)
  text = re.sub(r'π', 'pi', text)
  text = re.sub(r'\d+(\.\d+)?', '<NUM>', text)
  symb = r'[+\-*/=()<>]'
  text = re.sub(symb, lambda x: '' + x.group() + '', text)
  text = re.sub(r'[^\w\s<>\-+/=()*^]', ' ',text )
  text = re.sub(r'\s+', ' ', text).strip()
  text = text.lower()
  return text


In [ ]:
newregex('''Scientists were letting rats into a maze and observed whether they would reach the goal. They found that $56\%$ of the black rats and $84\%$ of the white rats reached the goal. The ratio of the number of black and white rats at the goal was $1:2$.

What was the ratio of the number of black and white rats at the start?

(M. Petrová)''')

'scientists were letting rats into a maze and observed whether they would reach the goal they found that <num> of the black rats and <num> of the white rats reached the goal the ratio of the number of black and white rats at the goal was <num> <num> what was the ratio of the number of black and white rats at the start (m petrová)'

In [ ]:
index = train['label'].value_counts().index
ratios =  train['label'].value_counts().values/train['label'].value_counts().sum() + 1e-10
pd.DataFrame(ratios, index=index)

,0
label,
0,0.256944
1,0.239376
5,0.179311
4,0.168024
2,0.101973
3,0.036117
6,0.009815
7,0.008440


**3. REGEX FOR GREEK LETTERS, NUMBERS AND SPECIAL CHARACTERS**

In [ ]:
def regex(text):
  spc_char = r"[^\w\s\u0370-\u03FF0-9]"
  combined = f"{spc_char}|\n"
  matches = re.compile(combined)
  return matches.sub(r'', text)

**3. REMOVE STOPWORDS**

In [ ]:
def remove_stopwords(text):
  new = []
  for i in text.split():
    if i in stopwords.words('english'):
      new.append('')
    else:
      new.append(i)
  x = new[:]
  new.clear()
  return " ".join(x)

**4. STEMMING**

In [ ]:
def stemmer(text):
  new = []
  for i in text.split():
    new.append(PorterStemmer().stem(i))
  x = new[:]
  new.clear()
  return " ".join(x)

**OR**

In [ ]:
def preprocessing(text):
  text = text.lower()
  greek_lets =  r"[\u0370-\u03FF]"
  nums = r"[0-9]"
  spc_char = r"[^\w\s\u0370-\u03FF0-9]"
  combined = f"{greek_lets}|{nums}|{spc_char}|\n"
  matches = re.compile(combined)
  text =  matches.sub(r'', text)
  new = []
  for i in text.split():
    if i in stopwords.words('english'):
      new.append('')
    else:
      new.append(i)
  x = new[:]
  new.clear()
  text =  " ".join(x)
  new2 = []
  for i in text.split():
    new2.append(PorterStemmer().stem(i))
  y = new2[:]
  new2.clear()
  text = " ".join(y)
  return text

# **FINAL TEXT PREPROCESSING**

In [ ]:
def maths_preprocessing(text):

  text = re.sub(r'α', 'alpha', text)
  text = re.sub(r'β', 'beta', text)
  text = re.sub(r'π', 'pi', text)
  text = re.sub(r'\d+(\.\d+)?', '<NUM>', text)
  symb = r'[+\-*/=()<>]'
  text = re.sub(symb, lambda x: '' + x.group() + '', text)
  text = re.sub(r'[^\w\s<>\-+/=*^]', ' ',text )
  text = re.sub(r'\s+', ' ', text).strip()
  text = text.lower()

  new = []
  for i in text.split():
    if i in stopwords.words('english'):
      new.append('')
    else:
      new.append(i)
  x = new[:]
  new.clear()
  text =  " ".join(x)
  new2 = []
  for i in text.split():
    new2.append(PorterStemmer().stem(i))
  y = new2[:]
  new2.clear()
  text = " ".join(y)
  return text

In [ ]:
train['newQuestion'] = train['Question'].apply(preprocessing)

In [ ]:
train['transformed'] = train['Question'].apply(maths_preprocessing)

In [ ]:
train['transformed'][456]

'<num> line segment connect two point sphere call chord sphere sphere radiu <num> length two chord ab cd <num> sqrt <num> <num> sqrt <num> respect n midpoint ab cd respect endpoint pair chord move surfac sphere given follow four proposit <num> chord ab cd may intersect point <num> chord ab cd may intersect point n <num> maximum valu mn <num> <num> minimum valu mn <num> among true proposit <num> <num> <num> b <num> <num> <num> c <num> <num> <num> <num> <num> <num>'

In [ ]:
train['newQuestion'][456]

'line segment connect two point sphere call chord spherefor sphere radiu length two chord ab cd sqrt sqrt respect n midpoint ab cd respect endpoint pair chord move surfac sphere given follow four proposit chord ab cd may intersect point chord ab cd may intersect point n maximum valu mn minimum valu mn among true proposit b c'

In [ ]:
train

,Question,label,newQuestion,transformed
0,A solitaire game is played as follows. Six di...,3,solitair game play follow six distinct pair ma...,solitair game play follow six distinct pair ma...
1,2. The school table tennis championship was he...,5,school tabl tenni championship held accord oly...,<num> school tabl tenni championship held acco...
2,"Given that $x, y,$ and $z$ are real numbers th...",0,given x z real number satisfi beginalign x sqr...,given x z real number satisfi begin align* x =...
3,$25 \cdot 22$ Given three distinct points $P\l...,1,cdot given three distinct point pleftx_ y_righ...,<num> cdot <num> given three distinct point p ...
4,I am thinking of a five-digit number composed ...,5,think fivedigit number compos even digit swap ...,think five-digit number compos even digit swap...
...,...,...,...,...
10184,Is there a first- or second-degree polynomial ...,0,first seconddegre polynomi take valu variabl v...,first- second-degre polynomi take valu <num> <...
10185,4. Let the side lengths of $\triangle A B C$ b...,1,let side length triangl b c x x maximum valu a...,<num> let side length triangl b c <num> x <num...
10186,A triangular array of squares has one square i...,5,triangular array squar one squar first row two...,triangular array squar one squar first row two...
10187,Suppose we are given seven points that are equ...,1,suppos given seven point equal space around ci...,suppos given seven point equal space around ci...


# **VECTORIZATION AND SMOTE**

In [ ]:
X = train['transformed']
y = train['label']

xtrain, xtest, ytrain, ytest = train_test_split(X,y, random_state=42, test_size=0.2)

tfidf = TfidfVectorizer()
x_train = tfidf.fit_transform(xtrain).toarray()
x_test = tfidf.transform(xtest).toarray()

smote = SMOTE()
x_train, y_train = smote.fit_resample(x_train, ytrain)

# **MODEL BUILDING**

In [ ]:
mnb = MultinomialNB(alpha = 0.01)
mnb.fit(x_train, y_train)
ypred = mnb.predict(x_test)
print(classification_report(ytest, ypred))
f1_score(ytest, ypred, average = 'micro')

              precision    recall  f1-score   support

           0       0.76      0.43      0.54       522
           1       0.87      0.87      0.87       500
           2       0.51      0.81      0.62       221
           3       0.49      0.60      0.54        63
           4       0.67      0.76      0.71       331
           5       0.66      0.58      0.62       362
           6       0.08      0.45      0.13        20
           7       0.75      0.79      0.77        19

    accuracy                           0.66      2038
   macro avg       0.60      0.66      0.60      2038
weighted avg       0.71      0.66      0.67      2038



0.6648675171736997

In [ ]:
mnb = GaussianNB()
mnb.fit(x_train, y_train)
ypred = mnb.predict(x_test)
print(classification_report(ytest, ypred))
f1_score(ytest, ypred, average = 'micro')

              precision    recall  f1-score   support

           0       0.49      0.11      0.18       522
           1       0.80      0.66      0.73       500
           2       0.31      0.68      0.42       221
           3       0.19      0.40      0.26        63
           4       0.56      0.55      0.56       331
           5       0.56      0.40      0.47       362
           6       0.04      0.50      0.07        20
           7       0.24      0.58      0.34        19

    accuracy                           0.45      2038
   macro avg       0.40      0.49      0.38      2038
weighted avg       0.55      0.45      0.46      2038



0.4479882237487733

In [ ]:
bnb = BernoulliNB(alpha = 0.01)
bnb.fit(x_train, y_train)
ypred = bnb.predict(x_test)
print(classification_report(ytest, ypred))
f1_score(ytest, ypred, average = 'micro')

              precision    recall  f1-score   support

           0       0.79      0.68      0.73       522
           1       0.88      0.86      0.87       500
           2       0.57      0.77      0.66       221
           3       0.85      0.46      0.60        63
           4       0.61      0.76      0.68       331
           5       0.68      0.58      0.63       362
           6       0.24      0.35      0.29        20
           7       0.75      0.63      0.69        19

    accuracy                           0.72      2038
   macro avg       0.67      0.64      0.64      2038
weighted avg       0.73      0.72      0.72      2038



0.7188420019627085

In [ ]:
from sklearn.ensemble import BaggingClassifier
ada = BaggingClassifier(n_estimators = 50, estimator=LogisticRegression(max_iter=1000))
ada.fit(x_train, y_train)
ypred = ada.predict(x_test)
print(classification_report(ytest, ypred))
f1_score(ytest, ypred, average = 'micro')

In [ ]:
dt = DecisionTreeClassifier(max_depth=64)
dt.fit(x_train, y_train)
dtpred = dt.predict(x_test)
print(classification_report(ytest, dtpred))
f1_score(ytest, dtpred, average="micro")

              precision    recall  f1-score   support

           0       0.68      0.64      0.66       522
           1       0.81      0.80      0.81       500
           2       0.53      0.55      0.54       221
           3       0.86      0.87      0.87        63
           4       0.63      0.67      0.65       331
           5       0.60      0.58      0.59       362
           6       0.27      0.45      0.34        20
           7       0.39      0.47      0.43        19

    accuracy                           0.67      2038
   macro avg       0.60      0.63      0.61      2038
weighted avg       0.67      0.67      0.67      2038



0.6678115799803729

In [ ]:
rf = RandomForestClassifier(n_estimators=200, max_depth=48)
rf.fit(x_train, y_train)
rfpred = rf.predict(x_test)
print(classification_report(ytest, rfpred))
f1_score(ytest, rfpred, average="micro")

              precision    recall  f1-score   support

           0       0.81      0.64      0.71       522
           1       0.87      0.92      0.90       500
           2       0.67      0.71      0.69       221
           3       0.85      0.84      0.85        63
           4       0.69      0.83      0.75       331
           5       0.68      0.65      0.67       362
           6       0.16      0.30      0.21        20
           7       0.71      0.79      0.75        19

    accuracy                           0.75      2038
   macro avg       0.68      0.71      0.69      2038
weighted avg       0.76      0.75      0.75      2038



0.7526987242394504

In [ ]:
lr = LogisticRegression(max_iter=1000)
lr.fit(x_train, y_train)
lrpred = lr.predict(x_test)
print(classification_report(ytest, lrpred))
f1_score(ytest, lrpred, average="micro")

              precision    recall  f1-score   support

           0       0.80      0.59      0.68       522
           1       0.90      0.89      0.89       500
           2       0.65      0.73      0.69       221
           3       0.70      0.83      0.76        63
           4       0.72      0.76      0.74       331
           5       0.68      0.69      0.68       362
           6       0.10      0.45      0.16        20
           7       0.59      0.89      0.71        19

    accuracy                           0.73      2038
   macro avg       0.64      0.73      0.66      2038
weighted avg       0.76      0.73      0.74      2038



0.7325809617271836

In [ ]:
# xg = XGBClassifier(n_estimators = 200, max_depth = 48)
# xg.fit(x_train, y_train)
# xgpred = xg.predict(x_test)
# print(classification_report(ytest, xgpred))
# f1_score(ytest, xgpred, average="micro")

In [ ]:
sv = SVC()
sv.fit(x_train, y_train)
svpred = sv.predict(x_test)
print(classification_report(ytest, svpred))
f1_score(ytest, svpred, average="micro")

              precision    recall  f1-score   support

           0       0.74      0.70      0.72       522
           1       0.89      0.89      0.89       500
           2       0.68      0.67      0.67       221
           3       0.95      0.57      0.71        63
           4       0.74      0.76      0.75       331
           5       0.68      0.72      0.70       362
           6       0.10      0.25      0.14        20
           7       0.92      0.63      0.75        19

    accuracy                           0.75      2038
   macro avg       0.71      0.65      0.67      2038
weighted avg       0.76      0.75      0.75      2038



0.7477919528949951

In [ ]:
estimators = [('rf', rf), ('bnb', bnb), ('sv', sv), ('lr', lr)]
vc = VotingClassifier(estimators=estimators)
vc.fit(x_train, y_train)
vcpred = vc.predict(x_test)
print(classification_report(ytest, vcpred))
f1_score(ytest, vcpred, average="micro")

              precision    recall  f1-score   support

           0       0.76      0.71      0.73       522
           1       0.88      0.92      0.90       500
           2       0.69      0.69      0.69       221
           3       0.85      0.73      0.79        63
           4       0.71      0.82      0.76       331
           5       0.75      0.65      0.70       362
           6       0.16      0.30      0.21        20
           7       0.79      0.79      0.79        19

    accuracy                           0.76      2038
   macro avg       0.70      0.70      0.70      2038
weighted avg       0.77      0.76      0.76      2038



0.7639842983316978

In [ ]:
estim = [('rf', rf), ('bnb', bnb), ('lr', lr)]
stack = StackingClassifier(estimators=estim, final_estimator=rf)
stack.fit(x_train, y_train)
preds = stack.predict(x_test)
print(classification_report(ytest, preds))
f1_score(ytest, preds, average='micro')

              precision    recall  f1-score   support

           0       0.77      0.70      0.74       522
           1       0.88      0.93      0.91       500
           2       0.73      0.70      0.71       221
           3       0.83      0.71      0.77        63
           4       0.71      0.78      0.74       331
           5       0.72      0.73      0.73       362
           6       0.12      0.20      0.15        20
           7       0.93      0.68      0.79        19

    accuracy                           0.77      2038
   macro avg       0.71      0.68      0.69      2038
weighted avg       0.77      0.77      0.77      2038



0.7693817468105987